In [1]:
from qwen2.modeling_qwen2 import Qwen2MLAForCausalLM
from transformers import AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

/root/miniconda3/envs/transmla/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = Qwen2MLAForCausalLM.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B", device_map='cuda:7', attn_implementation="sdpa", partial_rotary_factor=2)
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-3B")
model

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.86s/it]
Some weights of Qwen2MLAForCausalLM were not initialized from the model checkpoint at /root/mfx/huggingface/Qwen/Qwen2.5-3B and are newly initialized: ['model.layers.0.self_attn.k_up_proj.weight', 'model.layers.0.self_attn.v_up_proj.weight', 'model.layers.1.self_attn.k_up_proj.weight', 'model.layers.1.self_attn.v_up_proj.weight', 'model.layers.10.self_attn.k_up_proj.weight', 'model.layers.10.self_attn.v_up_proj.weight', 'model.layers.11.self_attn.k_up_proj.weight', 'model.layers.11.self_attn.v_up_proj.weight', 'model.layers.12.self_attn.k_up_proj.weight', 'model.layers.12.self_attn.v_up_proj.weight', 'model.layers.13.self_attn.k_up_proj.weight', 'model.layers.13.self_attn.v_up_proj.weight', 'model.layers.14.self_attn.k_up_proj.weight', 'model.layers.14.self_attn.v_up_proj.weight', 'model.layers.15.self_attn.k_up_proj.weight', 'model.layers.15.self_attn.v_up_proj.weight', 'model.layers.16.self_attn.k_up_proj.weight

Qwen2MLAForCausalLM(
  (model): Qwen2MLAModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaMLAttention(
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_up_proj): Linear(in_features=256, out_features=2048, bias=False)
          (v_up_proj): Linear(in_features=256, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()


In [3]:
hidden_size = model.config.hidden_size
n_heads = model.config.num_attention_heads
kv_heads = model.config.num_key_value_heads
head_dim = model.config.hidden_size//model.config.num_attention_heads
latent_dim = kv_heads * head_dim
kv_groups = model.config.num_attention_heads // model.config.num_key_value_heads
model.config.partial_rotary_factor

2

In [4]:
# Insert identity matrices
for name,module in model.named_modules():
    if 'k_up_proj' in name or "v_up_proj" in name:
        weight = torch.stack([torch.eye(latent_dim).reshape(kv_heads, head_dim, latent_dim)]*kv_groups,dim=1).reshape(hidden_size, latent_dim).contiguous().to(module.weight.data.device,module.weight.data.dtype)
        if 'k_up_proj' in name:
            weight = weight.view(hidden_size, kv_heads, head_dim).transpose(1,2).contiguous().view(hidden_size, latent_dim)
        module.weight.data=weight
    elif 'k_proj' in name:
        module.weight.data = module.weight.data.view(kv_heads, head_dim, hidden_size).transpose(0,1).contiguous().view(latent_dim, hidden_size)
        module.bias.data = module.bias.data.view(kv_heads, head_dim).transpose(0,1).contiguous().view(latent_dim)

In [5]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:7"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。一天，森林里来了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽看到米洛的痛苦，决定帮助它。

蓝羽用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个勇敢的朋友。”从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。

有一天，森林里发生了一场大火，许多动物都逃到了安全的地方。蓝羽和米洛也加入了逃跑的队伍。在逃跑的过程中，他们遇到了一只凶猛的野狼。野狼想要吃掉他们，但蓝羽和米洛并没有害怕。他们一起合作，用智慧和勇气打败了野狼。

从那以后，蓝羽和米洛变得更加勇敢和坚强。他们知道，只要有朋友在身边，就没有克服不了的困难。而他们的友谊也变得更加深厚，成为了森林中最美好的传说。<|endoftext|>


In [6]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Orthogonal q_proj and k_up_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(q_k_up, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->hdD',torch.sqrt(S),V) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            module.q_proj.bias.data = S_sqrtV[:,:,-1].reshape(-1).contiguous()
            S_sqrtV = S_sqrtV[:,:,:-1]
        module.k_up_proj.weight.data = US_sqrt.reshape(n_heads*head_dim, latent_dim).contiguous()
        module.q_proj.weight.data = S_sqrtV.reshape(n_heads*head_dim, hidden_size).contiguous()

        # Orthogonal o_proj and v_up_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim)
        v_up_o = torch.einsum("hdc,Dhd->hcD",v_up_weight, o_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(v_up_o, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->Dhd',torch.sqrt(S),V) # (hidden_size, n_heads, head_dim)
        module.v_up_proj.weight.data = US_sqrt.reshape(hidden_size, latent_dim).contiguous()
        module.o_proj.weight.data = S_sqrtV.reshape(hidden_size, n_heads*head_dim).contiguous()

In [7]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:7"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。一天，森林里来了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽看到米洛的痛苦，决定帮助它。

蓝羽用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个勇敢的朋友。”从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。

有一天，森林里发生了一场大火，许多动物都逃到了安全的地方。蓝羽和米洛也加入了逃跑的队伍。在逃跑的过程中，他们遇到了一只凶猛的野狼。野狼想要吃掉他们，但蓝羽和米洛并没有害怕。他们一起合作，用智慧和勇气打败了野狼。

从那以后，蓝羽和米洛变得更加勇敢和坚强。他们知道，只要有朋友在身边，就没有克服不了的困难。而他们的友谊也变得更加深厚，成为了森林中最美好的传说。<|endoftext|>


In [8]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Absorb k_up_proj into q_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        q_proj = torch.nn.Linear(hidden_size, n_heads*latent_dim, bias=(module.q_proj.bias is not None))
        q_proj = q_proj.to(device=module.q_proj.weight.device, dtype=module.q_proj.weight.dtype)
        if module.q_proj.bias is not None:
            q_proj.bias.data = q_k_up[:,:,-1].reshape(-1).contiguous()
            q_k_up = q_k_up[:,:,:-1]
        q_proj.weight.data = q_k_up.reshape(n_heads*latent_dim, hidden_size).contiguous()
        setattr(module, "q_proj", q_proj)
        delattr(module, "k_up_proj")
        # Absorb v_up_proj into o_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim) # (n_heads, head_dim, hidden_size)
        v_up_o = torch.einsum("hdc,Dhd->Dhc",v_up_weight, o_weight) # (hidden_size, n_heads, latent_dim), rank<=head_dim
        o_proj = torch.nn.Linear(n_heads*latent_dim, hidden_size, bias=(module.o_proj.bias is not None))
        o_proj = o_proj.to(device=module.o_proj.weight.device, dtype=module.o_proj.weight.dtype)
        o_proj.weight.data = v_up_o.reshape(hidden_size, n_heads*latent_dim).contiguous()
        if module.o_proj.bias is not None:
            o_proj.bias.data = module.o_proj.bias
        setattr(module, "o_proj", o_proj)
        delattr(module, "v_up_proj")
        module.absorb = True

In [9]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:7"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。一天，森林里来了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽看到米洛的痛苦，决定帮助它。

蓝羽用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个勇敢的朋友。”从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。

有一天，森林里发生了一场大火，许多动物都逃到了安全的地方。蓝羽和米洛也加入了逃跑的队伍。在逃跑的过程中，他们遇到了一只凶猛的野狼。野狼想要吃掉他们，但蓝羽和米洛并没有害怕。他们一起合作，用智慧和勇气打败了野狼。

从那以后，蓝羽和米洛变得更加勇敢和坚强。他们知道，只要有朋友在身边，就没有克服不了的困难。而他们的友谊也变得更加深厚，成为了森林中最美好的传说。<|endoftext|>


In [10]:
model.config.absorb=True
model.config.head_dim=256
model.config.latent_dim_factor=2
model.config.num_key_value_heads=1
model.config.partial_rotary_factor=1

In [11]:
model.config.absorb=True
model.save_pretrained("TransMLA_Qwen2.5-3B_absorb")
tokenizer.save_pretrained("TransMLA_Qwen2.5-3B_absorb")

[2025-02-19 00:52:59,757] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlvsym'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlopen'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlclose'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: unde

('TransMLA_Qwen2.5-3B_absorb/tokenizer_config.json',
 'TransMLA_Qwen2.5-3B_absorb/special_tokens_map.json',
 'TransMLA_Qwen2.5-3B_absorb/vocab.json',
 'TransMLA_Qwen2.5-3B_absorb/merges.txt',
 'TransMLA_Qwen2.5-3B_absorb/added_tokens.json',
 'TransMLA_Qwen2.5-3B_absorb/tokenizer.json')

In [12]:
from qwen2.modeling_qwen2 import Qwen2MLAForCausalLM
from transformers import AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

In [13]:
model = Qwen2MLAForCausalLM.from_pretrained("TransMLA_Qwen2.5-3B_absorb", device_map='cuda:7', attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("TransMLA_Qwen2.5-3B_absorb")
model

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


Qwen2MLAForCausalLM(
  (model): Qwen2MLAModel(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2MLAttention(
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (q_proj): Linear(in_features=2048, out_features=4096, bias=True)
          (o_proj): Linear(in_features=4096, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (no

In [14]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:7"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。一天，森林里来了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽看到米洛的痛苦，决定帮助它。

蓝羽用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个勇敢的朋友。”从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。

有一天，森林里发生了一场大火，许多动物都逃到了安全的地方。蓝羽和米洛也加入了逃跑的队伍。在逃跑的过程中，他们遇到了一只凶猛的野狼。野狼想要吃掉他们，但蓝羽和米洛并没有害怕。他们一起合作，用智慧和勇气打败了野狼。

从那以后，蓝羽和米洛变得更加勇敢和坚强。他们知道，只要有朋友在身边，就没有克服不了的困难。而他们的友谊也变得更加深厚，成为了森林中最美好的传说。<|endoftext|>


In [15]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [16]:
from vllm import ModelRegistry
from qwen2.vllm_qwen2 import Qwen2MLAForCausalLM
ModelRegistry.register_model("Qwen2MLAForCausalLM", Qwen2MLAForCausalLM)

2025-02-19 00:54:29,283	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [17]:
# Currently only support tensor_parallel_size=1
from vllm import LLM, SamplingParams
llm = LLM("TransMLA_Qwen2.5-3B_absorb", trust_remote_code=True)

INFO 02-19 00:54:29 config.py:2272] Downcasting torch.float32 to torch.float16.
INFO 02-19 00:54:29 config.py:510] This model supports multiple tasks: {'generate', 'embed', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 02-19 00:54:29 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='TransMLA_Qwen2.5-3B_absorb', speculative_config=None, tokenizer='TransMLA_Qwen2.5-3B_absorb', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time

Loading safetensors checkpoint shards:   0% Completed | 0/3 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  33% Completed | 1/3 [00:00<00:01,  1.26it/s]
Loading safetensors checkpoint shards:  67% Completed | 2/3 [00:01<00:00,  1.17it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 3/3 [00:02<00:00,  1.11it/s]



INFO 02-19 00:54:34 model_runner.py:1099] Loading model weights took 6.3619 GB
INFO 02-19 00:54:35 worker.py:241] Memory profiling takes 1.48 seconds
INFO 02-19 00:54:35 worker.py:241] the current vLLM instance can use total_gpu_memory (39.38GiB) x gpu_memory_utilization (0.90) = 35.44GiB
INFO 02-19 00:54:35 worker.py:241] model weights take 6.36GiB; non_torch_memory takes 0.10GiB; PyTorch activation peak memory takes 2.52GiB; the rest of the memory reserved for KV Cache is 26.46GiB.
INFO 02-19 00:54:35 gpu_executor.py:76] # GPU blocks: 48165, # CPU blocks: 7281
INFO 02-19 00:54:35 gpu_executor.py:80] Maximum concurrency for 32768 tokens per request: 23.52x
INFO 02-19 00:54:38 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utiliz

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.41it/s]

INFO 02-19 00:54:52 model_runner.py:1535] Graph capturing finished in 15 secs, took 0.23 GiB
INFO 02-19 00:54:52 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 18.25 seconds


In [22]:
prompts = [
    "给我讲一个故事吧",
]
sampling_params = SamplingParams(temperature=0, max_tokens=512)
outputs = llm.generate(prompts, sampling_params)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(prompt+generated_text)

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.43s/it, est. speed input: 2.06 toks/s, output: 104.28 toks/s]

给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事。

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽拥有一身美丽的蓝色羽毛，但它总是感到孤独，因为它没有朋友。一天，森林里来了一只受伤的小兔子，名叫米洛。米洛的腿受伤了，无法行走。蓝羽看到米洛的痛苦，决定帮助它。

蓝羽用它的翅膀轻轻地拍打着地面，帮助米洛站起来。米洛感激地看着蓝羽，说：“谢谢你，蓝羽。你真是个勇敢的朋友。”从那天起，蓝羽和米洛成了最好的朋友。他们一起探索森林，分享彼此的故事和秘密。

有一天，森林里发生了一场大火，许多动物都逃到了安全的地方。蓝羽和米洛也加入了逃跑的队伍。在逃跑的过程中，他们遇到了一只凶猛的野狼。野狼想要吃掉他们，但蓝羽和米洛并没有害怕。他们一起合作，用智慧和勇气打败了野狼。

从那以后，蓝羽和米洛变得更加勇敢和坚强。他们知道，只要有朋友在身边，就没有克服不了的困难。而他们的友谊也变得更加深厚，成为了森林中最美好的传说。
